# fast read of current operational forecast fst files

In [1]:
import numpy as np
import os
import re
import fstpy.all as fstpy
import pandas as pd

file_dir = '/fs/site5/eccc/prod/ops/mirror/suites/hrdps_national_20211130/n1/gridpt/prog/pres/'

hours = [(0,0),(0,24),(0,48),(6,30),(12,36),(18,42)]

sources = np.unique([f[0:8] for f in os.listdir(file_dir) if re.search(r'[0-9]{10}_[0-9]{3}', f)])

file_list = []

for f in sources:
    for hour in hours:
        file_list.append(f'{f}{hour[0]:02}_{hour[1]:03}')

dataframes = [fstpy.add_grid_column(fstpy.get_basic_dataframe(file_dir + f)) for f in os.listdir(file_dir) if f in file_list]

df = pd.concat([dataframes[0]],ignore_index=True)
df

,nomvar,typvar,etiket,ni,nj,nk,dateo,ip1,ip2,ip3,...,ig2,ig3,ig4,datev,lng,swa,key,path,shape,grid
0,HU,P,N1_V600_N,2540,1290,1,457352000,50,0,0,...,74520,1,0,457352000,404980,2335,1,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(2540, 1290)",7682674520
1,HU,P,N1_V600_N,2540,1290,1,457352000,30,0,0,...,74520,1,0,457352000,461214,204825,1025,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(2540, 1290)",7682674520
2,GZ,P,N1_V600_N,2540,1290,1,457352000,1000,0,0,...,74520,1,0,457352000,661416,435432,2049,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(2540, 1290)",7682674520
3,GZ,P,N1_V600_N,2540,1290,1,457352000,985,0,0,...,74520,1,0,457352000,656966,766140,3073,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(2540, 1290)",7682674520
4,WW,P,N1_V600_N,2540,1290,1,457352000,900,0,0,...,74520,1,0,457352000,903648,1094623,4097,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(2540, 1290)",7682674520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,>>,X,N1_V600_N,2540,1,1,457352000,76826,74520,1,...,500,56000,44000,457352000,2564,113754968,613377,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(2540, 1)",7682674520
344,^^,X,N1_V600_N,1,1290,1,457352000,76826,74520,1,...,500,56000,44000,457352000,1314,113756250,614401,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(1, 1290)",7682674520
345,!!,X,N1_V600_N,3,39,1,0,76826,74520,0,...,0,0,0,0,258,113756907,615425,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(3, 39)",7682674520
346,>>,X,N1_V600_N,2540,1,1,457352000,69862,45551,4,...,500,56000,44000,457352000,2564,113757036,616449,/fs/site5/eccc/prod/ops/mirror/suites/hrdps_na...,"(2540, 1)",6986245551


## Select clean TT

In [2]:
cols = fstpy.BASE_COLUMNS
if 'd' in cols:
    cols.remove('d')
tt_df = fstpy.select_with_meta(df,['TT'])
tt_df.loc[tt_df.nomvar.isin(['^^','>>']),'dateo'] = 0
tt_df = fstpy.metadata_cleanup(tt_df)
tt_df = fstpy.add_dask_column(tt_df)
tt_df = fstpy.add_decoded_date_column(tt_df,'datev')
tt_df.drop(columns='d')

,nomvar,typvar,etiket,ni,nj,nk,dateo,ip1,ip2,ip3,...,ip2_pkind,ip3_dec,ip3_kind,ip3_pkind,surface,follow_topography,ascending,interval,vctype,date_of_validity
0,^^,X,N1_V600_N,1,1290,1,0,76826,74520,1,...,,1.0,100,,False,False,True,None,UNKNOWN,2022-05-10
1,>>,X,N1_V600_N,2540,1,1,0,76826,74520,1,...,,1.0,100,,False,False,True,None,UNKNOWN,2022-05-10
2,!!,X,N1_V600_N,3,39,1,0,76826,74520,0,...,,0.0,100,,False,False,True,None,UNKNOWN,NaT
3,TT,P,N1_V600_N,2540,1290,1,457352000,1015,0,0,...,H,0.0,100,,False,False,False,None,PRESSURE_2001,2022-05-10
4,TT,P,N1_V600_N,2540,1290,1,457352000,1000,0,0,...,H,0.0,100,,False,False,False,None,PRESSURE_2001,2022-05-10
5,TT,P,N1_V600_N,2540,1290,1,457352000,20,0,0,...,H,0.0,100,,False,False,False,None,PRESSURE_2001,2022-05-10
6,TT,P,N1_V600_N,2540,1290,1,457352000,10,0,0,...,H,0.0,100,,False,False,False,None,PRESSURE_2001,2022-05-10
7,TT,P,N1_V600_N,2540,1290,1,457352000,970,0,0,...,H,0.0,100,,False,False,False,None,PRESSURE_2001,2022-05-10
8,TT,P,N1_V600_N,2540,1290,1,457352000,950,0,0,...,H,0.0,100,,False,False,False,None,PRESSURE_2001,2022-05-10
9,TT,P,N1_V600_N,2540,1290,1,457352000,450,0,0,...,H,0.0,100,,False,False,False,None,PRESSURE_2001,2022-05-10


In [3]:
import fstpy.all as fstpy
fstpy.get_grid_definition_params(tt_df)

Read(999) ^^   X  N1_V600_N           1    1290     1  457352000        76826     74520         1        0        0  E 32  E  1430   500 56000 44000
Read(999) >>   X  N1_V600_N        2540       1     1  457352000        76826     74520         1        0        0  E 32  E  1430   500 56000 44000


{'id': 0,
 'shape': (2540, 1290),
 'ni': 2540,
 'nj': 1290,
 'grtyp': 'Z',
 'ig1': 76826,
 'ig2': 74520,
 'ig3': 1,
 'ig4': 0,
 'grref': 'E',
 'ig1ref': 1430,
 'ig2ref': 500,
 'ig3ref': 56000,
 'ig4ref': 44000,
 'nsubgrids': 1,
 'subgridid': [0],
 'xg1': 53.0,
 'xg2': 260.0,
 'xg3': 50.0,
 'xg4': 275.0,
 'xlat1': 53.0,
 'xlon1': 260.0,
 'xlat2': 50.0,
 'xlon2': 275.0,
 'ax': array([[156.39749],
        [156.42   ],
        [156.44249],
        ...,
        [213.48   ],
        [213.50249],
        [213.525  ]], dtype=float32),
 'ay': array([[-12.302501, -12.280001, -12.257501, ...,  16.655   ,  16.6775  ,
          16.7     ]], dtype=float32),
 'lat0': 39.62603378649634,
 'lon0': 226.3704808223897,
 'dlat': 0.022500001894031226,
 'dlon': 0.02250000096156213,
 'rlat0': -12.30250072479248,
 'rlon0': 156.39749145507812,
 'tag1': 76826,
 'tag2': 74520,
 'tag3': 1}

## convert to xarray -> split netcdf

In [4]:
import fstd2nc
import xarray as xr
import fstpy.all as fstpy
write_path = '/home/sbf000/ss6/netcdf/'

meta_df = fstpy.select_meta(tt_df)
groups = tt_df.loc[tt_df.nomvar=='TT'].groupby(['grid','nomvar','date_of_validity','level'])
for (grid,nomvar,date_of_validity,level) , current_df in groups:
    current_tt_df = pd.concat([meta_df,current_df],ignore_index=True)
    file = f"{write_path}/{nomvar}_{date_of_validity.strftime('%Y%m%d%H')}_{level}.nc"
    ds = fstd2nc.Buffer.from_fstpy(current_tt_df).to_netcdf(file)

xr.open_dataset(file)

Read(998) !!   X  N1_V600_N           3      39     1  000000000        76826     74520         0        0        0  E 64  X  2001     0     0     0
Write(999) !!   X  N1_V600_N           3      39     1  000000000        76826     74520         0        0        0  E 64  X  2001     0     0     0
Read(998) >>   X  N1_V600_N        2540       1     1  457352000        76826     74520         1        0        0  E 32  E  1430   500 56000 44000
Write(999) >>   X  N1_V600_N        2540       1     1  000000000        76826     74520         1        0        0  E 32  E  1430   500 56000 44000
Read(998) ^^   X  N1_V600_N           1    1290     1  457352000        76826     74520         1        0        0  E 32  E  1430   500 56000 44000
Write(999) ^^   X  N1_V600_N           1    1290     1  000000000        76826     74520         1        0        0  E 32  E  1430   500 56000 44000
Found 1 RPN input file(s)
Read(999) !!   X  N1_V600_N           3      39     1  000000000        76826

<xarray.Dataset>
Dimensions:       (time: 1, pres: 1, rlat: 1290, rlon: 2540)
Coordinates:
  * time          (time) datetime64[ns] 2022-05-10
  * pres          (pres) float32 1.015e+03
  * rlat          (rlat) float32 -12.3 -12.28 -12.26 -12.24 ... 16.66 16.68 16.7
  * rlon          (rlon) float32 -14.82 -14.8 -14.78 ... 42.26 42.28 42.31
    leadtime      (time) timedelta64[ns] ...
    reftime       datetime64[ns] ...
    lon           (rlat, rlon) float32 ...
    lat           (rlat, rlon) float32 ...
Data variables:
    TT            (time, pres, rlat, rlon) float32 ...
    rotated_pole  |S1 ...
Attributes:
    Conventions:  CF-1.6

In [ ]:
import logging
from pathlib import Path
from typing import Tuple, Dict, List, Sequence, Union
import numpy as np

logger = logging.getLogger()
def write_geo_tiff(file_name: Union[str, Path], raster: np.array, swne: Sequence, no_data_value: Union[int, float] = None, dtype: str= None):
    '''
    Writes the raster in data['raster'] to a GeoTiff file.
    Args:
        file_name (str):    Name of the output file.
        raster (np.array): The raster. (0, 0) is the top-left (north-west) pixel.
                           The shape of the raster is (latitude, longitude).
        swne (tuple):      Coordinates of bottom-left and top-right pixels.
                           The convention is that these give the **centers** of
                           the pixels.
        no_data_value:       The no data value.
        dtype (GDAL type): GDAL data type (e.g. Byte, UInt16, Int16, UInt32, Int32, Float32, Float64)
    '''
    try:
        from osgeo import gdal
    except ImportError:
        logger.error('Unable to import \'gdal\'.')
        raise
    try:
        import numpy as np
    except ImportError:
        logger.error('Unable to import \'numpy\'.')
        raise
    
    # We cannot set the default dtype in the function signature since we cannot 
    # assume gdal to exist. So we set it here.
    if dtype is None:
        dtype = gdal.GDT_Float32
    elif dtype == 'Byte':
        dtype = gdal.GDT_Byte
    elif dtype == 'UInt16':
        dtype = gdal.GDT_UInt16
    elif dtype == 'Int16':
        dtype = gdal.GDT_Int16
    elif dtype == 'UInt32':
        dtype = gdal.GDT_UInt32
    elif dtype == 'Int32':
        dtype = gdal.GDT_Int32
    elif dtype == 'Float32':
        dtype = gdal.GDT_Float32
    elif dtype == 'Float64':
        dtype = gdal.GDT_Float64
    else:
        raise Exception(f'Data type {dtype} is not supported.')

    n_j, n_i = raster.shape
    if (n_j <= 1) or (n_i <= 1):
        raise Exception('GeoTiff writer does not support rasters of a single pixel\'s width or heigth.')
    lat_min, lon_min, lat_max, lon_max = swne
    d_lat = (lat_max - lat_min) / (n_j - 1)
    d_lon = (lon_max - lon_min) / (n_i - 1)
    nw_corner_lat = lat_max + d_lat / 2.
    nw_corner_lon = lon_min - d_lon / 2.
    gdal_geo_ref = (nw_corner_lon, d_lon, 0, nw_corner_lat, 0, -d_lat)
    if no_data_value is not None:
        raster = np.where(np.isnan(raster), no_data_value, raster)

    # Use GDAL to create the GeoTiff
    drv = gdal.GetDriverByName('GTiff')
    ds = drv.Create(str(file_name), n_i, n_j, 1, dtype)
    
    from osgeo import osr
    sr = osr.SpatialReference()
    sr.ImportFromEPSG(4326)
    ds.SetProjection(sr.ExportToWkt())

    ds.SetGeoTransform(gdal_geo_ref)
    raster_band = ds.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(raster)
    ds = None

In [5]:
import pendulum, datetime
import numpy as np
import json
from pathlib import Path
import logging

from mpl_toolkits.basemap import shiftgrid


logger = logging.getLogger()
upload_dir = Path('/home/sbf000/ss6/upload/')

if not upload_dir.is_dir():
    raise Exception(f'The upload directory {upload_dir} does not exist.')

# # https://confluence.ecmwf.int/display/CKB/How+to+plot+GRIB+files+with+Python+and+matplotlib
# # need to shift data grid longitudes from (0..360) to (-180..180)
# lons = np.linspace(float(grb['longitudeOfFirstGridPointInDegrees']), \
# float(grb['longitudeOfLastGridPointInDegrees']), int(grb['Ni']) )
# lats = np.linspace(float(grb['latitudeOfFirstGridPointInDegrees']), \
# float(grb['latitudeOfLastGridPointInDegrees']), int(grb['Nj']) )
# data, lons = shiftgrid(180., data, lons, start=False)
# grid_lon, grid_lat = np.meshgrid(lons, lats) #regularly spaced 2D grid

def shift_data(fisrt_lon,last_lon,ni,first_lat,lats_lat,nj):
    # need to shift data grid longitudes from (0..360) to (-180..180)
    lons = np.linspace(float(fisrt_lon), float(last_lon), int(ni) )
    lats = np.linspace(float(first_lat), float(lats_lat), int(nj) )
    data, lons = shiftgrid(180., data, lons, start=False)
    grid_lon, grid_lat = np.meshgrid(lons, lats) #regularly spaced 2D grid

def upload_to_pairs(file: Path) -> None:
        with xr.open_dataset(file) as xr_dataset:
            xr_array = xr_dataset.TT
            # display(xr_array.values.shape)
            time_string = xr_array.time.values[0].astype('datetime64[s]').astype(datetime.datetime).strftime('%Y%m%dT%H%M%SZ')
            level_string = f'{xr_array.pres.values[0]:.2f}{xr_array.pres.units}'
            # display(time_string)
            
            # display(xr_array.name)
            u_file = upload_dir / f'raster_{xr_array.name}_{level_string}_{time_string}.tiff'
            # display(u_file)
            swne = (
                np.min(xr_array['lat'].values),
                np.min(xr_array['lon'].values),
                np.max(xr_array['lat'].values),
                np.max(xr_array['lon'].values)
            )
            # display(swne)
            write_geo_tiff(u_file, np.squeeze(xr_array.values), swne, dtype='Float32')
            logger.debug(f'Created {u_file}.')
            
            m_file = u_file.with_suffix('.tiff.meta.json')
            display(m_file)
            url_str = str(os.path.basename(u_file))
            display(url_str)
            with m_file.open('w') as fp:
                json.dump(
                    {
                        'datalayer_id' : [xr_array.name],
                        'url' :  url_str,
                        'pairsdatatype' : '2draster',
                        'timestamp' : time_string,
                        'level' : level_string,
                        'ignoretile' : True,
                        'datainterpolation' : 'near',
                        'geospatialprojection' : 'EPSG:4326'
                    },
                    fp
                )
            logger.debug(f'Created {m_file}.')
            
                    
upload_to_pairs(file)                
    

PosixPath('/home/sbf000/ss6/upload/raster_TT_1015.00hPa_20220510T000000Z.tiff.meta.json')

'raster_TT_1015.00hPa_20220510T000000Z.tiff'

In [13]:
import rasterio
original_img = rasterio.open('/home/sbf000/ss6/operation.forecasts.lamgrib2.hrdps_national.continental/2mt_2019010100_000.tiff')
input_crs = original_img.crs
input_gt  = original_img.transform
display(input_crs)
display(input_gt)

CRS.from_wkt('PROJCS["unnamed",GEOGCS["Coordinate System imported from GRIB file",DATUM["unnamed",SPHEROID["Sphere",6371229,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",60],PARAMETER["central_meridian",252],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",SOUTH],AXIS["Northing",SOUTH]]')

Affine(2500.0, 0.0, -2099127.494496938,
       0.0, -2500.0, -2099388.521499629)